In [21]:
import polars as pl
from src.tank_ts_endpoint import fetch_tank_ts_data, GetTanksTimeDataReq

req = GetTanksTimeDataReq(scada_id='6941901')

df = await fetch_tank_ts_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

key_metric,ts,value,scada_id,tank_name,uom
object,datetime[μs],f64,str,str,str
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 04:11:36.465651,172.663947,"""6941901""","""OilTank1Volume""","""bbl"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 03:11:36.465651,172.645457,"""6941901""","""OilTank1Volume""","""bbl"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 02:11:36.465651,172.627082,"""6941901""","""OilTank1Volume""","""bbl"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 01:11:36.465651,172.608948,"""6941901""","""OilTank1Volume""","""bbl"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 00:11:36.465651,172.591192,"""6941901""","""OilTank1Volume""","""bbl"""
…,…,…,…,…,…
b17f11ac-d318-429a-b607-a5ba24b3d6a2,2024-03-02 10:11:36.465651,70.615716,"""6941901""","""OilTank1Level""","""in"""
b17f11ac-d318-429a-b607-a5ba24b3d6a2,2024-03-02 09:11:36.465651,68.968335,"""6941901""","""OilTank1Level""","""in"""
b17f11ac-d318-429a-b607-a5ba24b3d6a2,2024-03-02 08:11:36.465651,67.289497,"""6941901""","""OilTank1Level""","""in"""


In [22]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("tank_name").str.extract_groups(pattern))
lf = lf.unnest("separated_metrics")
lf.collect()

key_metric,ts,value,scada_id,tank_name,uom,is_esd,tank_type,tank_number,tank_metric
object,datetime[μs],f64,str,str,str,str,str,str,str
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 04:11:36.465651,172.663947,"""6941901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 03:11:36.465651,172.645457,"""6941901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 02:11:36.465651,172.627082,"""6941901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 01:11:36.465651,172.608948,"""6941901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 00:11:36.465651,172.591192,"""6941901""","""OilTank1Volume""","""bbl""",null,"""Oil""","""1""","""Volume"""
…,…,…,…,…,…,…,…,…,…
b17f11ac-d318-429a-b607-a5ba24b3d6a2,2024-03-02 10:11:36.465651,70.615716,"""6941901""","""OilTank1Level""","""in""",null,"""Oil""","""1""","""Level"""
b17f11ac-d318-429a-b607-a5ba24b3d6a2,2024-03-02 09:11:36.465651,68.968335,"""6941901""","""OilTank1Level""","""in""",null,"""Oil""","""1""","""Level"""
b17f11ac-d318-429a-b607-a5ba24b3d6a2,2024-03-02 08:11:36.465651,67.289497,"""6941901""","""OilTank1Level""","""in""",null,"""Oil""","""1""","""Level"""


In [23]:
lf = lf.drop("tank_name", "is_esd", "tank_type", "tank_number")
lf.collect()

key_metric,ts,value,scada_id,uom,tank_metric
object,datetime[μs],f64,str,str,str
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 04:11:36.465651,172.663947,"""6941901""","""bbl""","""Volume"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 03:11:36.465651,172.645457,"""6941901""","""bbl""","""Volume"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 02:11:36.465651,172.627082,"""6941901""","""bbl""","""Volume"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 01:11:36.465651,172.608948,"""6941901""","""bbl""","""Volume"""
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,2024-03-06 00:11:36.465651,172.591192,"""6941901""","""bbl""","""Volume"""
…,…,…,…,…,…
b17f11ac-d318-429a-b607-a5ba24b3d6a2,2024-03-02 10:11:36.465651,70.615716,"""6941901""","""in""","""Level"""
b17f11ac-d318-429a-b607-a5ba24b3d6a2,2024-03-02 09:11:36.465651,68.968335,"""6941901""","""in""","""Level"""
b17f11ac-d318-429a-b607-a5ba24b3d6a2,2024-03-02 08:11:36.465651,67.289497,"""6941901""","""in""","""Level"""


In [24]:
aggregated_lf = lf.group_by(["key_metric", "tank_metric", "uom"]).agg([
    pl.col("ts").alias("timestamps"),
    pl.col("value").alias("values")
])

aggregated_lf.collect()

key_metric,tank_metric,uom,timestamps,values
object,str,str,list[datetime[μs]],list[f64]
b17f11ac-d318-429a-b607-a5ba24b3d6a2,"""Level""","""in""","[2024-03-06 04:11:36.465651, 2024-03-06 03:11:36.465651, … 2024-03-02 06:11:36.465651]","[62.159021, 62.140531, … 63.878773]"
4ac0b7d9-55fe-4865-bc29-6c202ff6349e,"""Volume""","""bbl""","[2024-03-06 04:11:36.465651, 2024-03-06 03:11:36.465651, … 2024-03-02 06:11:36.465651]","[172.663947, 172.645457, … 174.383698]"
